# Regression Analysis: Predicting Continuous Values

**Interactive Notebook** - Section 1: Foundational Machine Learning

Welcome to your deep dive into regression analysis! In this notebook, you'll learn how to build models that predict continuous values like house prices, temperature, or sales figures.

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand the difference between classification and regression
- ✅ Master linear regression and its mathematical foundations
- ✅ Learn about polynomial regression and overfitting
- ✅ Implement regularization techniques (Lasso, Ridge)
- ✅ Evaluate regression models with appropriate metrics
- ✅ Apply regression to real-world datasets

## 📋 Prerequisites

- Completion of "01_Introduction_to_Machine_Learning.ipynb"
- Basic understanding of algebra and statistics
- Familiarity with Python and scikit-learn

**Estimated Time**: 2-3 hours

## 🔧 Setup and Installation

Let's set up our environment with the necessary libraries.

In [ ]:
# Install required packages
!pip install -q numpy pandas matplotlib seaborn scikit-learn ipywidgets

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing, make_regression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100)

print("✅ All libraries imported successfully!")

## 📊 Understanding Regression vs Classification

Let's start by understanding the key differences between regression and classification:

In [ ]:
# Create comparison table
ml_comparison = {
    'Aspect': [
        'Output Type',
        'Problem Type',
        'Examples',
        'Algorithms',
        'Evaluation Metrics',
        'Visualization'
    ],
    'Classification': [
        'Discrete categories/classes',
        'Predict which category',
        'Spam detection, Image recognition, Medical diagnosis',
        'Logistic Regression, Decision Trees, Random Forest, SVM',
        'Accuracy, Precision, Recall, F1-Score, AUC',
        'Confusion matrix, ROC curves'
    ],
    'Regression': [
        'Continuous numerical values',
        'Predict numerical value',
        'House prices, Temperature, Sales forecasting, Stock prices',
        'Linear Regression, Polynomial Regression, Gradient Boosting',
        'MSE, RMSE, MAE, R²',
        'Scatter plots, Residual plots'
    ]
}

df_comparison = pd.DataFrame(ml_comparison)
display(HTML("<h3>Classification vs Regression</h3>"))
display(df_comparison.style.background_gradient(cmap='RdYlGn'))

## 🏠 Dataset: California Housing Prices

Let's work with the California Housing dataset to predict house prices based on various features.

In [ ]:
# Load the California Housing dataset
housing = fetch_california_housing()
X = housing.data
y = housing.target
feature_names = housing.feature_names
target_name = housing.target_names[0]

# Create DataFrame
df_housing = pd.DataFrame(X, columns=feature_names)
df_housing[target_name] = y

print("🏠 California Housing Dataset Overview")
print(f"Number of samples: {len(df_housing)}")
print(f"Number of features: {len(feature_names)}")
print(f"Target: {target_name}")
print(f"\nFeatures: {feature_names}")

# Display dataset statistics
display(HTML("<h4>Dataset Statistics:</h4>"))
display(df_housing.describe().round(2))

# Display first few samples
display(HTML("<h4>Sample Data:</h4>"))
display(df_housing.head())

In [ ]:
# Explore the target variable distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(y, kde=True, bins=30)
plt.title('Distribution of House Prices')
plt.xlabel('Median House Value ($100,000s)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
sns.boxplot(y=y)
plt.title('Box Plot of House Prices')
plt.xlabel('Median House Value ($100,000s)')

plt.tight_layout()
plt.show()

print(f"📊 Target Variable Statistics:")
print(f"Mean: ${np.mean(y)*100000:,.0f}")
print(f"Median: ${np.median(y)*100000:,.0f}")
print(f"Standard Deviation: ${np.std(y)*100000:,.0f}")
print(f"Min: ${np.min(y)*100000:,.0f}")
print(f"Max: ${np.max(y)*100000:,.0f}")

In [ ]:
# Explore correlations
plt.figure(figsize=(12, 8))
correlation_matrix = df_housing.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Find features most correlated with target
target_correlations = correlation_matrix[target_name].drop(target_name).sort_values(ascending=False)
print("🔗 Feature Correlations with House Prices:")
for feature, correlation in target_correlations.items():
    print(f"{feature}: {correlation:.3f}")

## 📈 Linear Regression: The Foundation

Linear regression is the simplest regression algorithm. It assumes a linear relationship between features and the target variable.

In [ ]:
# Data preparation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("📊 Data Preparation Complete")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]}")

In [ ]:
# Train Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred = lr_model.predict(X_train_scaled)
y_test_pred = lr_model.predict(X_test_scaled)

# Evaluate the model
def evaluate_regression(y_true, y_pred, dataset_name="Dataset"):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"📊 {dataset_name} Performance:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")
    
    return mse, rmse, mae, r2

print("🎯 Linear Regression Results")
print("="*50)
train_mse, train_rmse, train_mae, train_r2 = evaluate_regression(y_train, y_train_pred, "Training")
print()
test_mse, test_rmse, test_mae, test_r2 = evaluate_regression(y_test, y_test_pred, "Testing")

# Display coefficients
print("\n📈 Model Coefficients:")
coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': lr_model.coef_
}).sort_values('Coefficient', ascending=False)

for idx, row in coefficients.iterrows():
    print(f"{row['Feature']}: {row['Coefficient']:.4f}")

print(f"\nIntercept: {lr_model.intercept_:.4f}")

In [ ]:
# Visualize predictions vs actual values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, alpha=0.6, s=20)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title(f'Training Set (R² = {train_r2:.3f})')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred, alpha=0.6, s=20)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title(f'Testing Set (R² = {test_r2:.3f})')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze residuals
residuals = y_test - y_test_pred

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.scatter(y_test_pred, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
sns.histplot(residuals, kde=True, bins=30)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
from scipy import stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Residual Analysis:")
print(f"Mean of residuals: {np.mean(residuals):.4f}")
print(f"Standard deviation of residuals: {np.std(residuals):.4f}")
print(f"Skewness: {stats.skew(residuals):.4f}")
print(f"Kurtosis: {stats.kurtosis(residuals):.4f}")

## 🎛️ Interactive Linear Regression Explorer

Let's create an interactive tool to explore how linear regression works with different features:

In [ ]:
# Interactive linear regression explorer
feature1_dropdown = widgets.Dropdown(
    options=feature_names,
    value='MedInc',  # Most correlated feature
    description='Feature 1:'
)

feature2_dropdown = widgets.Dropdown(
    options=['None'] + list(feature_names),
    value='AveRooms',
    description='Feature 2:'
)

show_plane_checkbox = widgets.Checkbox(
    value=False,
    description='Show Regression Plane (3D)'
)

explore_button = widgets.Button(description='Explore Regression', button_style='info')
explore_output = widgets.Output()

def explore_regression(b):
    with explore_output:
        explore_output.clear_output()
        
        # Get selected features
        feat1 = feature1_dropdown.value
        feat2 = feature2_dropdown.value
        
        # Prepare data
        if feat2 == 'None':
            # Simple linear regression
            X_simple = df_housing[[feat1]].values
            y_simple = df_housing[target_name].values
            
            # Fit model
            lr_simple = LinearRegression()
            lr_simple.fit(X_simple, y_simple)
            y_pred_simple = lr_simple.predict(X_simple)
            
            # Calculate R²
            r2_simple = r2_score(y_simple, y_pred_simple)
            
            # Plot
            plt.figure(figsize=(10, 6))
            plt.scatter(X_simple, y_simple, alpha=0.6, s=20, label='Data')
            plt.plot(X_simple, y_pred_simple, 'r-', linewidth=2, label='Regression Line')
            plt.xlabel(feat1)
            plt.ylabel(target_name)
            plt.title(f'Simple Linear Regression (R² = {r2_simple:.3f})')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
            
            print(f"📈 Simple Linear Regression Results:")
            print(f"Feature: {feat1}")
            print(f"Coefficient: {lr_simple.coef_[0]:.4f}")
            print(f"Intercept: {lr_simple.intercept_:.4f}")
            print(f"R²: {r2_simple:.3f}")
            
        else:
            # Multiple linear regression
            X_multi = df_housing[[feat1, feat2]].values
            y_multi = df_housing[target_name].values
            
            # Fit model
            lr_multi = LinearRegression()
            lr_multi.fit(X_multi, y_multi)
            y_pred_multi = lr_multi.predict(X_multi)
            
            # Calculate R²
            r2_multi = r2_score(y_multi, y_pred_multi)
            
            if show_plane_checkbox.value:
                # 3D plot with regression plane
                from mpl_toolkits.mplot3d import Axes3D
                
                fig = plt.figure(figsize=(12, 8))
                ax = fig.add_subplot(111, projection='3d')
                
                # Create meshgrid for plane
                x1_range = np.linspace(X_multi[:, 0].min(), X_multi[:, 0].max(), 20)
                x2_range = np.linspace(X_multi[:, 1].min(), X_multi[:, 1].max(), 20)
                X1_grid, X2_grid = np.meshgrid(x1_range, x2_range)
                
                # Calculate plane
                Y_grid = (lr_multi.coef_[0] * X1_grid + 
                         lr_multi.coef_[1] * X2_grid + 
                         lr_multi.intercept_)
                
                # Plot
                ax.scatter(X_multi[:, 0], X_multi[:, 1], y_multi, alpha=0.6, s=20)
                ax.plot_surface(X1_grid, X2_grid, Y_grid, alpha=0.3, color='red')
                ax.set_xlabel(feat1)
                ax.set_ylabel(feat2)
                ax.set_zlabel(target_name)
                ax.set_title(f'Multiple Linear Regression (R² = {r2_multi:.3f})')
                
                plt.show()
            else:
                # 2D plot with color mapping
                plt.figure(figsize=(10, 8))
                scatter = plt.scatter(X_multi[:, 0], X_multi[:, 1], c=y_multi, alpha=0.6, s=30, cmap='viridis')
                plt.colorbar(scatter, label=target_name)
                plt.xlabel(feat1)
                plt.ylabel(feat2)
                plt.title(f'Multiple Linear Regression Features (R² = {r2_multi:.3f})')
                plt.grid(True, alpha=0.3)
                plt.show()
            
            print(f"📈 Multiple Linear Regression Results:")
            print(f"Features: {feat1}, {feat2}")
            print(f"Coefficient ({feat1}): {lr_multi.coef_[0]:.4f}")
            print(f"Coefficient ({feat2}): {lr_multi.coef_[1]:.4f}")
            print(f"Intercept: {lr_multi.intercept_:.4f}")
            print(f"R²: {r2_multi:.3f}")

explore_button.on_click(explore_regression)

print("🎛️ Interactive Linear Regression Explorer")
print("Select features to explore their relationship with house prices!")
display(widgets.VBox([
    feature1_dropdown, feature2_dropdown, show_plane_checkbox, 
    explore_button, explore_output
]))

## 📊 Polynomial Regression: Capturing Non-linear Relationships

Sometimes the relationship between features and target isn't linear. Let's explore polynomial regression!

In [ ]:
# Create synthetic non-linear data
np.random.seed(42)
X_synthetic = np.linspace(0, 10, 100)
y_synthetic = 2 * X_synthetic**2 - 3 * X_synthetic + 1 + np.random.normal(0, 2, 100)

X_synthetic = X_synthetic.reshape(-1, 1)

# Fit different polynomial degrees
degrees = [1, 2, 3, 5]
colors = ['blue', 'green', 'orange', 'red']

plt.figure(figsize=(15, 10))

for i, degree in enumerate(degrees):
    plt.subplot(2, 2, i + 1)
    
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly_features.fit_transform(X_synthetic)
    
    # Fit linear regression
    lr_poly = LinearRegression()
    lr_poly.fit(X_poly, y_synthetic)
    
    # Predict
    y_pred = lr_poly.predict(X_poly)
    
    # Calculate R²
    r2 = r2_score(y_synthetic, y_pred)
    
    # Plot
    plt.scatter(X_synthetic, y_synthetic, alpha=0.6, s=30, label='Data')
    plt.plot(X_synthetic, y_pred, color=colors[i], linewidth=2, label=f'Degree {degree}')
    plt.xlabel('X')
    plt.ylabel('y')
    plt.title(f'Polynomial Regression (Degree {degree}, R² = {r2:.3f})')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Polynomial Regression Analysis:")
print("Higher degree polynomials can fit the training data better,")
print("but may lead to overfitting!")

In [ ]:
# Apply polynomial regression to housing data
# Let's focus on the most correlated feature
best_feature = target_correlations.index[0]  # MedInc
X_best = df_housing[[best_feature]].values
y_best = df_housing[target_name].values

# Split data
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(
    X_best, y_best, test_size=0.3, random_state=42
)

# Try different polynomial degrees
degrees = [1, 2, 3, 4, 5]
train_scores = []
test_scores = []

for degree in degrees:
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly_transformed = poly_features.fit_transform(X_train_poly)
    X_test_poly_transformed = poly_features.transform(X_test_poly)
    
    # Fit model
    lr_poly = LinearRegression()
    lr_poly.fit(X_train_poly_transformed, y_train_poly)
    
    # Calculate scores
    train_r2 = lr_poly.score(X_train_poly_transformed, y_train_poly)
    test_r2 = lr_poly.score(X_test_poly_transformed, y_test_poly)
    
    train_scores.append(train_r2)
    test_scores.append(test_r2)

# Plot learning curves
plt.figure(figsize=(10, 6))
plt.plot(degrees, train_scores, 'o-', linewidth=2, markersize=8, label='Training R²')
plt.plot(degrees, test_scores, 'o-', linewidth=2, markersize=8, label='Testing R²')
plt.xlabel('Polynomial Degree')
plt.ylabel('R² Score')
plt.title('Polynomial Degree vs Model Performance')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Find best degree
best_degree = degrees[np.argmax(test_scores)]
print(f"🏆 Best polynomial degree: {best_degree}")
print(f"Best testing R²: {max(test_scores):.3f}")

# Compare with linear regression
lr_simple = LinearRegression()
lr_simple.fit(X_train_poly, y_train_poly)
simple_r2 = lr_simple.score(X_test_poly, y_test_poly)
print(f"Linear regression R²: {simple_r2:.3f}")
print(f"Improvement with polynomial: {max(test_scores) - simple_r2:.3f}")

## 🛡️ Regularization: Preventing Overfitting

Regularization helps prevent overfitting by adding a penalty term to the loss function. Let's explore L1 (Lasso) and L2 (Ridge) regularization.

In [ ]:
# Compare regularization techniques
alphas = [0.001, 0.01, 0.1, 1, 10, 100]

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5)
}

# Train and evaluate models
results = {}

print("🏆 Regularization Model Comparison")
print("="*60)

for name, model in models.items():
    # Train model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'mse': mse,
        'r2': r2,
        'model': model
    }
    
    print(f"{name}:")
    print(f"  MSE: {mse:.4f}")
    print(f"  R²: {r2:.4f}")
    print()

# Find best model
best_model_name = min(results.keys(), key=lambda x: results[x]['mse'])
print(f"🏆 Best Model: {best_model_name} (MSE: {results[best_model_name]['mse']:.4f})")

In [ ]:
# Explore regularization strength impact
ridge_train_scores = []
ridge_test_scores = []
lasso_train_scores = []
lasso_test_scores = []
lasso_coefficients = []

for alpha in alphas:
    # Ridge
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    ridge_train_scores.append(ridge.score(X_train_scaled, y_train))
    ridge_test_scores.append(ridge.score(X_test_scaled, y_test))
    
    # Lasso
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)
    lasso_train_scores.append(lasso.score(X_train_scaled, y_train))
    lasso_test_scores.append(lasso.score(X_test_scaled, y_test))
    lasso_coefficients.append(lasso.coef_)

# Plot regularization impact
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.semilogx(alphas, ridge_train_scores, 'o-', linewidth=2, markersize=8, label='Training')
plt.semilogx(alphas, ridge_test_scores, 'o-', linewidth=2, markersize=8, label='Testing')
plt.xlabel('Alpha (log scale)')
plt.ylabel('R² Score')
plt.title('Ridge Regularization')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.semilogx(alphas, lasso_train_scores, 'o-', linewidth=2, markersize=8, label='Training')
plt.semilogx(alphas, lasso_test_scores, 'o-', linewidth=2, markersize=8, label='Testing')
plt.xlabel('Alpha (log scale)')
plt.ylabel('R² Score')
plt.title('Lasso Regularization')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
lasso_coefficients = np.array(lasso_coefficients)
for i, feature in enumerate(feature_names):
    plt.semilogx(alphas, lasso_coefficients[:, i], 'o-', linewidth=2, markersize=6, label=feature)
plt.xlabel('Alpha (log scale)')
plt.ylabel('Coefficient Value')
plt.title('Lasso Coefficients')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Regularization Analysis:")
print("- Ridge: Shrinks coefficients but rarely sets them to zero")
print("- Lasso: Can set coefficients to zero (feature selection)")
print("- Higher alpha = stronger regularization = simpler models")

## 🎛️ Interactive Model Comparison

Let's create an interactive tool to compare different regression models:

In [ ]:
# Interactive model comparison
model_selector = widgets.SelectMultiple(
    options=['Linear Regression', 'Ridge', 'Lasso', 'ElasticNet', 'Random Forest'],
    value=['Linear Regression', 'Ridge', 'Lasso'],
    description='Models:'
)

test_size_slider = widgets.FloatSlider(
    value=0.3, min=0.1, max=0.5, step=0.05, description='Test Size:'
)

cross_val_checkbox = widgets.Checkbox(
    value=True,
    description='Use Cross-Validation'
)

compare_button = widgets.Button(description='Compare Models', button_style='success')
comparison_output = widgets.Output()

def compare_models_interactive(b):
    with comparison_output:
        comparison_output.clear_output()
        
        # Split data
        X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(
            X, y, test_size=test_size_slider.value, random_state=42
        )
        
        # Scale features
        scaler_cv = StandardScaler()
        X_train_cv_scaled = scaler_cv.fit_transform(X_train_cv)
        X_test_cv_scaled = scaler_cv.transform(X_test_cv)
        
        # Initialize models
        model_dict = {
            'Linear Regression': LinearRegression(),
            'Ridge': Ridge(alpha=1.0),
            'Lasso': Lasso(alpha=1.0),
            'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5),
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
        }
        
        selected_models = [model_dict[name] for name in model_selector.value]
        
        # Compare models
        results = {}
        
        print(f"🎯 Model Comparison Results")
        print(f"Test Size: {test_size_slider.value:.0%}")
        print(f"Cross-Validation: {cross_val_checkbox.value}")
        print("="*60)
        
        for i, (model, name) in enumerate(zip(selected_models, model_selector.value)):
            if cross_val_checkbox.value:
                # Use cross-validation
                cv_scores = cross_val_score(model, X_train_cv_scaled, y_train_cv, cv=5, scoring='r2')
                mean_score = cv_scores.mean()
                std_score = cv_scores.std()
                
                # Train on full training set for final evaluation
                model.fit(X_train_cv_scaled, y_train_cv)
                test_score = model.score(X_test_cv_scaled, y_test_cv)
                
                results[name] = {
                    'cv_mean': mean_score,
                    'cv_std': std_score,
                    'test_score': test_score
                }
                
                print(f"{name}:")
                print(f"  CV R²: {mean_score:.4f} (±{std_score:.4f})")
                print(f"  Test R²: {test_score:.4f}")
            else:
                # Simple train-test split
                model.fit(X_train_cv_scaled, y_train_cv)
                train_score = model.score(X_train_cv_scaled, y_train_cv)
                test_score = model.score(X_test_cv_scaled, y_test_cv)
                
                results[name] = {
                    'train_score': train_score,
                    'test_score': test_score
                }
                
                print(f"{name}:")
                print(f"  Train R²: {train_score:.4f}")
                print(f"  Test R²: {test_score:.4f}")
            
            print()
        
        # Visualize results
        plt.figure(figsize=(12, 6))
        
        if cross_val_checkbox.value:
            model_names = list(results.keys())
            cv_means = [results[name]['cv_mean'] for name in model_names]
            cv_stds = [results[name]['cv_std'] for name in model_names]
            test_scores = [results[name]['test_score'] for name in model_names]
            
            x_pos = np.arange(len(model_names))
            
            plt.bar(x_pos - 0.2, cv_means, 0.4, yerr=cv_stds, label='CV Score', alpha=0.7)
            plt.bar(x_pos + 0.2, test_scores, 0.4, label='Test Score', alpha=0.7)
            
            plt.xlabel('Models')
            plt.ylabel('R² Score')
            plt.title('Model Comparison with Cross-Validation')
            plt.xticks(x_pos, model_names, rotation=45)
            plt.legend()
        else:
            model_names = list(results.keys())
            train_scores = [results[name]['train_score'] for name in model_names]
            test_scores = [results[name]['test_score'] for name in model_names]
            
            x_pos = np.arange(len(model_names))
            
            plt.bar(x_pos - 0.2, train_scores, 0.4, label='Training Score', alpha=0.7)
            plt.bar(x_pos + 0.2, test_scores, 0.4, label='Test Score', alpha=0.7)
            
            plt.xlabel('Models')
            plt.ylabel('R² Score')
            plt.title('Model Comparison (Train vs Test)')
            plt.xticks(x_pos, model_names, rotation=45)
            plt.legend()
        
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

compare_button.on_click(compare_models_interactive)

print("🎛️ Interactive Model Comparison")
print("Select models to compare and explore their performance!")
display(widgets.VBox([
    model_selector, test_size_slider, cross_val_checkbox, 
    compare_button, comparison_output
]))

## 🏆 Challenge Exercises

Test your understanding with these regression challenges:

In [ ]:
# Challenge 1: Feature Engineering
print("🏆 Challenge 1: Feature Engineering")
print("="*50)
print("Task: Create new features to improve model performance")

# Create new features
df_engineered = df_housing.copy()

# Interaction features
df_engineered['rooms_per_person'] = df_engineered['AveRooms'] / df_engineered['AveOccup']
df_engineered['bedrooms_per_room'] = df_engineered['AveBedrms'] / df_engineered['AveRooms']
df_engineered['income_per_room'] = df_engineered['MedInc'] / df_engineered['AveRooms']

# Polynomial features for most important feature
df_engineered['MedInc_squared'] = df_engineered['MedInc'] ** 2
df_engineered['MedInc_cubed'] = df_engineered['MedInc'] ** 3

# Binning features
df_engineered['age_category'] = pd.cut(df_engineered['HouseAge'], 
                                     bins=[0, 10, 30, 52], 
                                     labels=['New', 'Middle-aged', 'Old'])

print("✅ New features created:")
print("- rooms_per_person: Average rooms per person")
print("- bedrooms_per_room: Bedroom to room ratio")
print("- income_per_room: Income per room")
print("- MedInc_squared/cubed: Polynomial income features")
print("- age_category: House age categories")

# Prepare data
df_engineered = pd.get_dummies(df_engineered, drop_first=True)
X_engineered = df_engineered.drop(target_name, axis=1).values
y_engineered = df_engineered[target_name].values

# Split and scale
X_train_eng, X_test_eng, y_train_eng, y_test_eng = train_test_split(
    X_engineered, y_engineered, test_size=0.3, random_state=42
)

scaler_eng = StandardScaler()
X_train_eng_scaled = scaler_eng.fit_transform(X_train_eng)
X_test_eng_scaled = scaler_eng.transform(X_test_eng)

# Train models
lr_original = LinearRegression()
lr_engineered = LinearRegression()

lr_original.fit(X_train_scaled, y_train)
lr_engineered.fit(X_train_eng_scaled, y_train_eng)

# Evaluate
original_score = lr_original.score(X_test_scaled, y_test)
engineered_score = lr_engineered.score(X_test_eng_scaled, y_test_eng)

print(f"\n📊 Performance Comparison:")
print(f"Original features R²: {original_score:.4f}")
print(f"Engineered features R²: {engineered_score:.4f}")
print(f"Improvement: {(engineered_score - original_score):.4f}")

if engineered_score > original_score:
    print("🎉 Feature engineering improved model performance!")
else:
    print("🤔 Original features performed better. Feature engineering requires careful experimentation!")

In [ ]:
# Challenge 2: Hyperparameter Tuning
print("🏆 Challenge 2: Hyperparameter Tuning")
print("="*50)
print("Task: Find optimal hyperparameters for Ridge regression")

# Define parameter grid
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

# Grid search
ridge = Ridge()
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

print(f"🏆 Best parameters: {grid_search.best_params_}")
print(f"🏆 Best CV score: {grid_search.best_score_:.4f}")

# Evaluate on test set
best_ridge = grid_search.best_estimator_
test_score = best_ridge.score(X_test_scaled, y_test)

print(f"🏆 Test score: {test_score:.4f}")

# Compare with default Ridge
default_ridge = Ridge(alpha=1.0)
default_ridge.fit(X_train_scaled, y_train)
default_score = default_ridge.score(X_test_scaled, y_test)

print(f"\n📊 Comparison:")
print(f"Default Ridge R²: {default_score:.4f}")
print(f"Tuned Ridge R²: {test_score:.4f}")
print(f"Improvement: {(test_score - default_score):.4f}")

# Visualize grid search results
plt.figure(figsize=(10, 6))
results_df = pd.DataFrame(grid_search.cv_results_)
pivot_table = results_df.pivot_table(
    values='mean_test_score', 
    index='param_alpha', 
    columns='param_solver'
)

sns.heatmap(pivot_table, annot=True, cmap='YlOrRd', fmt='.3f')
plt.title('Grid Search Results: Ridge Regression')
plt.xlabel('Solver')
plt.ylabel('Alpha')
plt.tight_layout()
plt.show()

In [ ]:
# Challenge 3: Real-world Application
print("🏆 Challenge 3: Sales Forecasting")
print("="*50)
print("Task: Apply regression techniques to a sales forecasting problem")

# Create synthetic sales data
np.random.seed(42)
n_months = 60

# Time series with trend and seasonality
time = np.arange(n_months)
trend = 0.5 * time
seasonality = 20 * np.sin(2 * np.pi * time / 12)  # Annual seasonality
noise = np.random.normal(0, 10, n_months)

sales = 100 + trend + seasonality + noise

# Create features
sales_data = pd.DataFrame({
    'month': time + 1,
    'sales': sales,
    'trend': trend,
    'seasonality': seasonality
})

# Add lag features
for lag in [1, 2, 3, 6, 12]:
    sales_data[f'sales_lag_{lag}'] = sales_data['sales'].shift(lag)

# Add rolling features
sales_data['sales_ma_3'] = sales_data['sales'].rolling(3).mean()
sales_data['sales_ma_6'] = sales_data['sales'].rolling(6).mean()

# Drop missing values
sales_data = sales_data.dropna()

print("📈 Sales Forecasting Dataset")
display(sales_data.head())

# Prepare features and target
feature_cols = ['month', 'trend', 'seasonality', 'sales_lag_1', 'sales_lag_2', 
               'sales_lag_3', 'sales_ma_3', 'sales_ma_6']

X_sales = sales_data[feature_cols].values
y_sales = sales_data['sales'].values

# Split data (time series split)
split_point = int(0.8 * len(X_sales))
X_train_sales = X_sales[:split_point]
X_test_sales = X_sales[split_point:]
y_train_sales = y_sales[:split_point]
y_test_sales = y_sales[split_point:]

# Scale features
scaler_sales = StandardScaler()
X_train_sales_scaled = scaler_sales.fit_transform(X_train_sales)
X_test_sales_scaled = scaler_sales.transform(X_test_sales)

# Train models
models_sales = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

results_sales = {}

for name, model in models_sales.items():
    model.fit(X_train_sales_scaled, y_train_sales)
    y_pred_sales = model.predict(X_test_sales_scaled)
    
    mse = mean_squared_error(y_test_sales, y_pred_sales)
    mae = mean_absolute_error(y_test_sales, y_pred_sales)
    r2 = r2_score(y_test_sales, y_pred_sales)
    
    results_sales[name] = {'mse': mse, 'mae': mae, 'r2': r2, 'predictions': y_pred_sales}
    
    print(f"{name}:")
    print(f"  MSE: {mse:.2f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  R²: {r2:.3f}")
    print()

# Visualize predictions
plt.figure(figsize=(12, 6))
plt.plot(sales_data['month'][split_point:], y_test_sales, 'b-', linewidth=2, label='Actual')
plt.plot(sales_data['month'][split_point:], results_sales['Random Forest']['predictions'], 
         'r--', linewidth=2, label='Random Forest Prediction')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.title('Sales Forecasting: Actual vs Predicted')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("🎉 Sales forecasting complete! Time series regression can be powerful for business applications.")

## 📋 Key Takeaways

### What You've Learned:

✅ **Regression vs Classification**: Understanding when to predict continuous values vs categories

✅ **Linear Regression**: The foundation of regression with mathematical interpretation

✅ **Model Evaluation**: MSE, RMSE, MAE, and R² metrics for regression problems

✅ **Residual Analysis**: Checking model assumptions and diagnostics

✅ **Polynomial Regression**: Capturing non-linear relationships

✅ **Regularization**: Preventing overfitting with Ridge, Lasso, and ElasticNet

✅ **Feature Engineering**: Creating new features to improve performance

✅ **Hyperparameter Tuning**: Finding optimal model parameters

✅ **Real-world Applications**: Sales forecasting and time series regression

### Key Concepts:

- **Overfitting**: When models perform well on training data but poorly on test data
- **Regularization**: Adding penalty terms to prevent overfitting
- **Cross-validation**: Robust evaluation technique for model selection
- **Feature importance**: Understanding which features drive predictions
- **Model diagnostics**: Residual analysis to check model assumptions

## 🚀 Next Steps

### Continue Your Learning Journey:

1. **📚 Next Notebook**: "03_Advanced_Classification_Techniques.ipynb" - Explore sophisticated classification methods

2. **🎯 Practice Challenges**: Try regression on different datasets from sklearn or Kaggle

3. **🌐 Real Data**: Download datasets from UCI Machine Learning Repository

4. **📖 Recommended Reading":
   - "Introduction to Statistical Learning" - Chapter 3 on Linear Regression
   - "The Elements of Statistical Learning" - Advanced regression techniques

5. **🛠️ Advanced Topics to Explore**:
   - Gradient Boosting Machines (XGBoost, LightGBM)
   - Support Vector Regression
   - Neural Networks for regression
   - Time series forecasting methods (ARIMA, Prophet)

### Quick Self-Assessment:

Can you explain:
- The difference between MSE, MAE, and RMSE?
- Why we need regularization in regression models?
- How to interpret R² score?
- What residual analysis tells us about model quality?
- When to use polynomial regression vs linear regression?

---

**🎉 Congratulations!** You've completed the Regression Analysis notebook! You now have solid skills in predicting continuous values and understanding regression model behavior.

**Remember**: Regression is a fundamental skill in machine learning with countless real-world applications. Keep practicing with different datasets and problems!